In [ ]:
import os
import pandas as pd
import re
import math
import copy
import numpy as np
import sys
# Add the root directory /workspaces/llm_etl to sys.path
sys.path.append(os.path.abspath(os.path.join('..', '..')))
# Now import your module
from spider2_utils import load_csv_database



-setup-

In [ ]:
import pandas as pd
_database = load_csv_database("modern_data", rows_limit=-1)
pizza_clean_customer_orders = _database["pizza_clean_customer_orders"]
pizza_clean_runner_orders = _database["pizza_clean_runner_orders"]

### Question
Calculate the total income from Meat Lovers pizzas priced at $12 and Vegetarian pizzas at $10. Include any extra toppings charged at $1 each. Ensure that canceled orders are filtered out. How much money has Pizza Runner earned in total?

### User Intent 1: Split 'extras' field by ',' and flatten into (order_id, each_extra)

In [ ]:
def split_extras(row):
    if pd.isna(row['extras']):
        return []
    extras = [extra.strip() for extra in row['extras'].split(',') if extra.strip()]
    return [{'order_id': row['order_id'], 'each_extra': extra} for extra in extras]

exploded_extras = pizza_clean_customer_orders.apply(split_extras, axis=1)
exploded_extras = pd.DataFrame([item for sublist in exploded_extras for item in sublist])

### User Intent 2: Count total extras per order_id

In [ ]:
get_extras_count = exploded_extras.groupby('order_id').size().reset_index(name='total_extras')

### User Intent 3: Filter out cancelled orders from runner orders

In [ ]:
valid_runner_orders = pizza_clean_runner_orders[pizza_clean_runner_orders['cancellation'].isna()]

### User Intent 4: Join customer and runner orders on 'order_id'

In [ ]:
orders_joined = pd.merge(pizza_clean_customer_orders, valid_runner_orders, on='order_id', how='inner')

### User Intent 5: Left join with total_extras per order_id

In [ ]:
orders_with_extras = pd.merge(orders_joined, get_extras_count, on='order_id', how='left')
orders_with_extras['total_extras'] = orders_with_extras['total_extras'].fillna(0).astype(int)

### User Intent 6: Calculate total price based on pizza_id

In [ ]:
orders_with_extras['price'] = orders_with_extras['pizza_id'].map({1: 12, 2: 10})

### User Intent 7: Group by to get total_price and extras per order

In [ ]:
calculate_totals = orders_with_extras.groupby(['order_id', 'pizza_id', 'total_extras'], as_index=False)[['price']].sum()
calculate_totals.rename(columns={'price': 'total_price'}, inplace=True)

### User Intent 8: Calculate total income as sum of price and extras

In [ ]:
total_income = calculate_totals['total_price'].sum() + calculate_totals['total_extras'].sum()
print('Total Income:', total_income)